# Hyperparamter tuning

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

if 'google.colab' in str(get_ipython()):
    !pip install keras_tuner
import keras_tuner as kt

import numpy as np
import pandas as pd
from IPython import display
from matplotlib import pyplot as plt

import os

print(tf.__version__)

2.18.0


## Importación y pretratamiento de datos
En esta demostración se va a utilizar de nuevo la base de datos de la central de ciclo combinado.

In [10]:
if 'google.colab' in str(get_ipython()):
  dataset_path = 'https://raw.githubusercontent.com/cursos-COnCEPT/curso-tensorflow/refs/heads/main/CCP.csv'
else:
  dataset_path = os.getcwd() + '\\CCP.csv'

raw_dataset = pd.read_csv(dataset_path)
dataset = raw_dataset.copy()
dataset.head()

# Split data
train_ratio = .7
val_ratio = .15
df = dataset.sample(frac=train_ratio+val_ratio, random_state=5)
df_test = dataset.drop(df.index)
df_train = df.sample(frac=val_ratio/(val_ratio+train_ratio), random_state=5)
df_valid = df.drop(df_train.index)

# Split features and target
X_train = df_train.drop('PE', axis=1)
X_valid = df_valid.drop('PE', axis=1)
X_test = df_test.drop('PE', axis=1)
y_train = df_train['PE']
y_valid = df_valid['PE']
y_test = df_test['PE']

# Scale to [0, 1]
max_ = X_train.max(axis=0)
min_ = X_train.min(axis=0)
X_train = (X_train - min_) / (max_ - min_)
X_valid = (X_valid - min_) / (max_ - min_)
X_test = (X_test - min_) / (max_ - min_)

## 1- Definición del modelo

Cuando se construye un modelo para *hypertuning*, se definen tanto el espacio de búsqueda de hiperparámetros como la arquitectura del modelo. El modelo que considera rangos de valores para los hiperparámetros en lugar de valores concretos se denomina *hypermodel*.

Aunque hay diferentes alternativas para construir un *hypermodel*, aquí lo haremos utilizando una función, que devolverá un modelo ya compilado.

A continuación, ajusta el número de neuronas de la primera capa, que puede estar comprendido entre 16 y 128, variando en potencias de base 2. Además, escoge el valor óptimo para el learning rate entre las siguientes opciones [0.001, 0.005, 0.0001].

In [11]:
def model_builder(hp):
    hp_neurons = hp.Int('neurons', 
                        min_value=16, 
                        max_value=128, 
                        step=2, 
                        sampling='log')
    hp_learning_rate = hp.Choice('learning_rate', 
                                values=[1e-3, 5e-4, 1e-4])

    model = keras.Sequential([
        layers.Input(shape=[X_train.shape[1]]),
        layers.Dense(hp_neurons, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1),
    ])

    optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
    
    model.compile(optimizer=optimizer,loss='mse',metrics=['mae'])

    return model

## 2- Ajuste de hiperparámetros

En Keras hay cuatro métodos diferentes para realizar la búsqueda - `RandomSearch`, `Hyperband`, `BayesianOptimization`, y `Sklearn`. En este ejemplo se utiliza el método `Hyperband`. 

Este algoritmo es rápido y eficiente porque prueba varias combinaciones de opciones (como el número de capas o el *learning rate*) y asigna más recursos (tiempo de entrenamiento) a las opciones que están funcionando mejor, para así encontrar la mejor solución en menos tiempo.

Para instanciar el *tuner*, hay que especificar el hipermodelo, el objetivo a optimizar y el número máximo de épocas a entrenar. 

In [12]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=100,
                     overwrite=True)
                     
tuner.search_space_summary()

Search space summary
Default search space size: 2
neurons (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 2, 'sampling': 'log'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0005, 0.0001], 'ordered': True}


Crea un callback tipo `EarlyStopping` antes de efectuar la búsqueda.

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss', # what to track
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

Efectúa la búsqueda, conocida como *hyperparameter tuning* con el comando `tuner.search`. Esta función utiliza los mismos argumentos que el método `model.fit`.

In [14]:
tuner.search(X_train, y_train,
            epochs=100,
            validation_data=(X_valid, y_valid), 
            batch_size=256,
            callbacks=[early_stopping],
            verbose='auto')

# Extraer la mejor combinación de hiperparámetros
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
El número de neuronas óptimo en la primera capa es {best_hps.get('neurons')} \n y el valor óptimo para el learning rate es {best_hps.get('learning_rate')}.
""")

Trial 12 Complete [00h 00m 04s]
val_loss: 206860.671875

Best val_loss So Far: 206560.78125
Total elapsed time: 00h 00m 48s

El número de neuronas óptimo en la primera capa es 64 
 y el valor óptimo para el learning rate es 0.001.



## 3- Entrenamiento y evaluación del modelo

Crea una nueva instancia del modelo utilizando la mejor configuración de los hiperparámetros y lleva a cabo el entrenamiento.

In [15]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, 
                    epochs=200,
                    validation_data=(X_valid, y_valid), 
                    batch_size=256,
                    callbacks=[early_stopping],
                    verbose='auto')

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - loss: 205523.0781 - mae: 453.0265 - val_loss: 206863.6250 - val_mae: 454.5027
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 205435.7031 - mae: 452.9235 - val_loss: 206812.7188 - val_mae: 454.4469
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 205565.9219 - mae: 453.0538 - val_loss: 206759.0469 - val_mae: 454.3877
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 205190.9062 - mae: 452.6545 - val_loss: 206696.0000 - val_mae: 454.3183
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 204905.0938 - mae: 452.3299 - val_loss: 206617.1875 - val_mae: 454.2312
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 205514.8281 - mae: 453.0082 - val_loss: 206520.5781 - val_mae: 454.1245
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 204853.8438 - mae: 452.2743 - val_loss: 206406.0156 - val_mae: 453.9978
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 205482.7500 - mae: 452.9627 - v

A continuación, evalúa la calidad del ajuste del modelo.

In [16]:
eval_result = model.evaluate(X_test, y_test)
print("MAE:", eval_result)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2899.7339 - mae: 44.8074
MAE: [2805.009521484375, 43.70571517944336]


## Conclusión
Este ejercicio no ha sido más que una introducción muy breve al tuneado de hiperparámetros. Para aprender más sobre `Keras Tuner`, puedes consultar estos recursos:

* [Keras Tuner en el blog de TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* [Sitio web del Sintonizador Keras](https://keras-team.github.io/keras-tuner/)